## Parameters

In [1]:
endpoint_url = 'http://rook-ceph-rgw-s3a.openshift-storage.svc.cluster.local'
aws_access_key_id = 'XQ0RN77YAHN23EMOAR5B'
aws_secret_access_key = 'NBgFuh139Rv0xHLY6LmYKSWvPytqmBFsJgAtJaRW'
bucket_base_name = 'xraylab-0001'
namespace = 'xraylab-0001'

## Imports

In [2]:
import boto3
import json
import botocore
import argparse

## S3 and SNS connections

In [3]:
s3 = boto3.client('s3',
                endpoint_url = endpoint_url,
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key = aws_secret_access_key,
                region_name = 'default',
                config=botocore.client.Config(signature_version = 's3'))

sns = boto3.client('sns', 
                endpoint_url = endpoint_url, 
                aws_access_key_id = aws_access_key_id,
                aws_secret_access_key= aws_secret_access_key,
                region_name='default', 
                config=botocore.client.Config(signature_version = 's3'))

## Create buckets

In [8]:
def create_bucket(bucket_name):
    result = s3.create_bucket(Bucket=bucket_name)
    return result

In [9]:
create_bucket(bucket_base_name)
create_bucket(bucket_base_name+'-processed')
create_bucket(bucket_base_name+'-anonymized')

{'ResponseMetadata': {'RequestId': 'tx000000000000000000376-005f8082f6-85e1-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000376-005f8082f6-85e1-s3a',
   'content-length': '0',
   'date': 'Fri, 09 Oct 2020 15:34:14 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}

### Verification

In [4]:
for bucket in s3.list_buckets()['Buckets']:
    print(bucket['Name'])

xraylab-0001
xraylab-0001-anonymized
xraylab-0001-processed


In [11]:
# Make bucket public read
for bucket in s3.list_buckets()['Buckets']:
    bucket_policy = {
                      "Version":"2012-10-17",
                      "Statement":[
                        {
                          "Sid":"AddPerm",
                          "Effect":"Allow",
                          "Principal": "*",
                          "Action":["s3:GetObject"],
                          "Resource":["arn:aws:s3:::{0}/*".format(bucket['Name'])]
                        }
                      ]
                    }
    bucket_policy = json.dumps(bucket_policy)
    s3.put_bucket_policy(Bucket=bucket['Name'], Policy=bucket_policy)

## Bucket Notifications configuration

In [5]:
attributes = {}
attributes['push-endpoint'] = 'kafka://my-cluster-kafka-bootstrap.'+namespace+':9092'
attributes['kafka-ack-level'] = 'broker'

In [6]:
def create_topic(topic):
    topic_arn = sns.create_topic(Name=topic, Attributes=attributes)['TopicArn']
    return topic_arn

In [7]:
create_topic('xray-images')

'arn:aws:sns:s3a::xray-images'

In [8]:
sns.list_topics()

{'Topics': [{'TopicArn': 'arn:aws:sns:s3a::xray-images'}],
 'ResponseMetadata': {'RequestId': '1c914c63-07e3-4079-ad3d-0d78ced9e705.135196.529',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000211-005f84b7d9-2101c-s3a',
   'content-type': 'application/xml',
   'content-length': '889',
   'date': 'Mon, 12 Oct 2020 20:08:57 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}

In [9]:
bucket_notifications_configuration = {
            "TopicConfigurations": [
                {
                    "Id": 'xray-images',
                    "TopicArn": 'arn:aws:sns:s3a::xray-images',
                    "Events": ["s3:ObjectCreated:*"]
                }
            ]
        }

s3.put_bucket_notification_configuration(Bucket = bucket_base_name,
        NotificationConfiguration=bucket_notifications_configuration)

{'ResponseMetadata': {'RequestId': 'tx000000000000000000213-005f84b7df-2101c-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000213-005f84b7df-2101c-s3a',
   'content-length': '0',
   'date': 'Mon, 12 Oct 2020 20:09:03 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0}}

In [10]:
s3.get_bucket_notification_configuration(Bucket = bucket_base_name)

{'ResponseMetadata': {'RequestId': 'tx000000000000000000214-005f84b7e2-2101c-s3a',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-request-id': 'tx000000000000000000214-005f84b7e2-2101c-s3a',
   'content-type': 'application/xml',
   'content-length': '192',
   'date': 'Mon, 12 Oct 2020 20:09:06 GMT',
   'connection': 'Keep-Alive'},
  'RetryAttempts': 0},
 'TopicConfigurations': [{'Id': 'xray-images',
   'TopicArn': 'arn:aws:sns:s3a::xray-images',
   'Events': ['s3:ObjectCreated:*']}]}

In [ ]:
sns.delete_topic(TopicArn='arn:aws:sns:ocs-storagecluster-cephobjectstore::xray-images')